In [54]:
from pathlib import Path
import re
import numpy as np
from typing import Dict, List, Tuple, Optional

In [55]:
# Root directory (relative to the current notebook)
ROOT = Path("./PoseErrors")
APE_DIR = ROOT / "APE"
RPE_DIR = ROOT / "RPE"

# Short-Full code in the file name (one-to-one correspondence with the full name)
short_tags: List[str] = ["orb", "akz", "kaz", "sft", "bsk", "spp"]
full_names: List[str] = ["ORB", "AKAZE", "KAZE", "SIFT", "BRISK", "SuperPoint"]

# Decimal places: display precision of mean±std in LaTeX
PREC = 3

# Unit scaling: 1=meter, 100=centimeter, 1000=millimeters
SCALE = 1000.0
UNIT_LABEL = "mm"   # "mm" or "m" etc.

# Whether to only accept combinations consisting entirely of short_tags; if False, unknown tags will be displayed in uppercase fallback
STRICT_TAGS = True

# Generate mapping
TAG2NAME: Dict[str, str] = dict(zip(short_tags, full_names))


In [56]:
def parse_20_runs_metrics(txt_path: Path) -> Dict[str, List[float]]:
    got = {"mean": [], "median": [], "rmse": []}
    if not txt_path.is_file():
        return got

    key_pat = re.compile(r"^\s*(mean|median|rmse)\s+([+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?)\s*$", re.I)

    with txt_path.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            m = key_pat.match(line)
            if m:
                k = m.group(1).lower()
                v = float(m.group(2))
                got[k].append(v)

    return got


def aggregate_mean_std(vals: List[float]) -> Tuple[Optional[float], Optional[float], int]:
    n = len(vals)
    if n == 0:
        return (None, None, 0)
    arr = np.asarray(vals, float)
    m = float(np.mean(arr))
    sd = float(np.std(arr, ddof=1)) if n > 1 else 0.0
    return (m, sd, n)


def collect_dir(dir_path: Path, dataset_prefix: str, tag2name: Dict[str, str], strict: bool) -> Dict[str, dict]:
    rows = {}
    for p in sorted(dir_path.glob(f"{dataset_prefix}*.txt")):
        combo = p.stem[len(dataset_prefix):]  # e.g. "akz_kaz"
        tags = [t for t in combo.split("_") if t]

        if strict and any(t not in tag2name for t in tags):
            continue

        label_parts = [(tag2name.get(t, t.upper())) for t in tags]
        label = " + ".join(label_parts) if label_parts else combo.upper()

        got = parse_20_runs_metrics(p)
        m_mean, s_mean, n_mean = aggregate_mean_std(got["mean"])
        m_median, s_median, n_median = aggregate_mean_std(got["median"])
        m_rmse, s_rmse, n_rmse = aggregate_mean_std(got["rmse"])

        rows[combo] = {
            "label": label,
            "metrics": {
                "mean":   {"mean": m_mean,   "std": s_mean,   "n": n_mean},
                "median": {"mean": m_median, "std": s_median, "n": n_median},
                "rmse":   {"mean": m_rmse,   "std": s_rmse,   "n": n_rmse},
            },
            "tags": tags,
        }
    return rows


def merge_ape_rpe(ape_rows: Dict[str, dict], rpe_rows: Dict[str, dict]) -> List[dict]:
    all_keys = sorted(set(ape_rows.keys()) | set(rpe_rows.keys()))
    out = []
    for k in all_keys:
        label = (ape_rows.get(k) or rpe_rows.get(k))["label"]
        tags  = (ape_rows.get(k) or rpe_rows.get(k))["tags"]
        out.append({
            "key": k,
            "label": label,
            "tags": tags,
            "APE": ape_rows.get(k, {}).get("metrics") if k in ape_rows else None,
            "RPE": rpe_rows.get(k, {}).get("metrics") if k in rpe_rows else None,
        })
    out.sort(key=lambda r: (len(r["tags"]), r["label"]))
    return out


In [57]:
def fmt_cell(m: Optional[float], s: Optional[float], prec: int, bold: bool) -> str:
    if m is None or s is None:
        return "--"
    m *= SCALE
    s *= SCALE
    txt = f"{m:.{prec}f} $\\pm$ {s:.{prec}f}"
    return f"\\textbf{{{txt}}}" if bold else txt


def build_latex_table(rows: List[dict], prec: int = 5) -> str:
    col_vals = {
        "A_mean": [],
        "A_rmse": [],
        "R_mean": [],
        "R_rmse": [],
    }
    for i, r in enumerate(rows):
        A = r["APE"]; R = r["RPE"]
        if A:
            if A["mean"]["mean"] is not None: col_vals["A_mean"].append((i, A["mean"]["mean"]))
            if A["rmse"]["mean"] is not None: col_vals["A_rmse"].append((i, A["rmse"]["mean"]))
        if R:
            if R["mean"]["mean"] is not None: col_vals["R_mean"].append((i, R["mean"]["mean"]))
            if R["rmse"]["mean"] is not None: col_vals["R_rmse"].append((i, R["rmse"]["mean"]))

    col_argmin = {k: (min(v, key=lambda t: t[1])[0] if v else None) for k, v in col_vals.items()}

    header = fr"""\begin{{table}}[htbp]
\centering
\caption{{Comparison of different inpainting regularizations under US and LA}}
\vspace{{0.2cm}}
\label{{tab:ape_rpe}}
\begin{{tabular}}{{lcccc}}
\toprule
\textbf{{Feature Extractor(s)}} &
\multicolumn{{2}}{{c}}{{\textbf{{Average Pose Error ({UNIT_LABEL})}}}} &
\multicolumn{{2}}{{c}}{{\textbf{{Relative Pose Error ({UNIT_LABEL}/f)}}}}  \\
\cmidrule(lr){{2-3}} \cmidrule(lr){{4-5}}
& Mean $\downarrow$ &  RMSE $\downarrow$ & Mean $\downarrow$  & RMSE $\downarrow$ \\
\midrule
"""
    body_lines = []
    for i, r in enumerate(rows):
        A = r["APE"]; R = r["RPE"]
        A_mean = fmt_cell(A["mean"]["mean"], A["mean"]["std"], prec, bold=(col_argmin["A_mean"] == i)) if A else "--"
        A_rmse = fmt_cell(A["rmse"]["mean"], A["rmse"]["std"], prec, bold=(col_argmin["A_rmse"] == i)) if A else "--"
        R_mean = fmt_cell(R["mean"]["mean"], R["mean"]["std"], prec, bold=(col_argmin["R_mean"] == i)) if R else "--"
        R_rmse = fmt_cell(R["rmse"]["mean"], R["rmse"]["std"], prec, bold=(col_argmin["R_rmse"] == i)) if R else "--"

        line = f"{r['label']}\n& {A_mean} & {A_rmse} & {R_mean} & {R_rmse} \\\\"
        body_lines.append(line)

    footer = r"""
\bottomrule
\end{tabular}
\end{table}
""".strip("\n")

    return header + "\n".join(body_lines) + "\n" + footer


In [58]:
# Dataset prefix (e.g. fr1 -> file name format is fr1_akz_kaz.txt)
DATASET = "fr1"
DATASET_PREFIX = f"{DATASET}_"

ape_rows = collect_dir(APE_DIR, DATASET_PREFIX, TAG2NAME, STRICT_TAGS)
rpe_rows = collect_dir(RPE_DIR, DATASET_PREFIX, TAG2NAME, STRICT_TAGS)
merged = merge_ape_rpe(ape_rows, rpe_rows)

latex_table = build_latex_table(merged, prec=PREC)
print(latex_table)

\begin{table}[htbp]
\centering
\caption{Comparison of different inpainting regularizations under US and LA}
\vspace{0.2cm}
\label{tab:ape_rpe}
\begin{tabular}{lcccc}
\toprule
\textbf{Feature Extractor(s)} &
\multicolumn{2}{c}{\textbf{Average Pose Error (mm)}} &
\multicolumn{2}{c}{\textbf{Relative Pose Error (mm/f)}}  \\
\cmidrule(lr){2-3} \cmidrule(lr){4-5}
& Mean $\downarrow$ &  RMSE $\downarrow$ & Mean $\downarrow$  & RMSE $\downarrow$ \\
\midrule
AKAZE
& 8.497 $\pm$ 0.382 & 10.029 $\pm$ 0.471 & 4.531 $\pm$ 0.056 & 5.455 $\pm$ 0.113 \\
BRISK
& 11.390 $\pm$ 1.972 & 13.475 $\pm$ 2.492 & 5.015 $\pm$ 0.112 & 6.176 $\pm$ 0.295 \\
KAZE
& 10.263 $\pm$ 1.343 & 12.176 $\pm$ 1.683 & 4.715 $\pm$ 0.112 & 5.793 $\pm$ 0.281 \\
ORB
& 19.033 $\pm$ 29.120 & 22.424 $\pm$ 31.245 & 5.652 $\pm$ 1.635 & 7.578 $\pm$ 2.676 \\
SIFT
& 8.719 $\pm$ 0.283 & 10.393 $\pm$ 0.353 & 4.810 $\pm$ 0.104 & 5.913 $\pm$ 0.187 \\
SuperPoint
& 22.241 $\pm$ 12.949 & 26.121 $\pm$ 15.415 & 5.166 $\pm$ 0.533 & 7.170 $\pm$ 1.533 

In [59]:
# Dataset prefix (e.g. fr1 -> file name format is fr1_akz_kaz.txt)
DATASET = "fr2"
DATASET_PREFIX = f"{DATASET}_"

ape_rows = collect_dir(APE_DIR, DATASET_PREFIX, TAG2NAME, STRICT_TAGS)
rpe_rows = collect_dir(RPE_DIR, DATASET_PREFIX, TAG2NAME, STRICT_TAGS)
merged = merge_ape_rpe(ape_rows, rpe_rows)

latex_table = build_latex_table(merged, prec=PREC)
print(latex_table)

\begin{table}[htbp]
\centering
\caption{Comparison of different inpainting regularizations under US and LA}
\vspace{0.2cm}
\label{tab:ape_rpe}
\begin{tabular}{lcccc}
\toprule
\textbf{Feature Extractor(s)} &
\multicolumn{2}{c}{\textbf{Average Pose Error (mm)}} &
\multicolumn{2}{c}{\textbf{Relative Pose Error (mm/f)}}  \\
\cmidrule(lr){2-3} \cmidrule(lr){4-5}
& Mean $\downarrow$ &  RMSE $\downarrow$ & Mean $\downarrow$  & RMSE $\downarrow$ \\
\midrule
AKAZE
& 15.397 $\pm$ 8.616 & 19.202 $\pm$ 10.615 & 2.192 $\pm$ 0.271 & 4.388 $\pm$ 1.510 \\
BRISK
& 15.824 $\pm$ 9.352 & 19.671 $\pm$ 10.930 & 2.698 $\pm$ 0.502 & 5.439 $\pm$ 1.902 \\
KAZE
& 21.720 $\pm$ 14.351 & 27.425 $\pm$ 17.914 & 2.158 $\pm$ 0.190 & 4.864 $\pm$ 1.830 \\
ORB
& 16.162 $\pm$ 11.069 & 20.763 $\pm$ 14.305 & 2.050 $\pm$ 0.387 & 4.043 $\pm$ 2.023 \\
SIFT
& 12.641 $\pm$ 8.295 & 16.572 $\pm$ 10.647 & 2.240 $\pm$ 0.366 & 4.086 $\pm$ 1.564 \\
SuperPoint
& 33.487 $\pm$ 10.415 & 41.583 $\pm$ 12.814 & 1.792 $\pm$ 0.153 & 4.651 $\pm$

In [61]:
# Unit scaling: 1=meter, 100=centimeter, 1000=millimeters
SCALE = 100.0
UNIT_LABEL = "cm"   # "mm" or "m" etc.

# Dataset prefix (e.g. fr1 -> file name format is fr1_akz_kaz.txt)
DATASET = "fr3"
DATASET_PREFIX = f"{DATASET}_"

ape_rows = collect_dir(APE_DIR, DATASET_PREFIX, TAG2NAME, STRICT_TAGS)
rpe_rows = collect_dir(RPE_DIR, DATASET_PREFIX, TAG2NAME, STRICT_TAGS)
merged = merge_ape_rpe(ape_rows, rpe_rows)

latex_table = build_latex_table(merged, prec=PREC)
print(latex_table)

\begin{table}[htbp]
\centering
\caption{Comparison of different inpainting regularizations under US and LA}
\vspace{0.2cm}
\label{tab:ape_rpe}
\begin{tabular}{lcccc}
\toprule
\textbf{Feature Extractor(s)} &
\multicolumn{2}{c}{\textbf{Average Pose Error (cm)}} &
\multicolumn{2}{c}{\textbf{Relative Pose Error (cm/f)}}  \\
\cmidrule(lr){2-3} \cmidrule(lr){4-5}
& Mean $\downarrow$ &  RMSE $\downarrow$ & Mean $\downarrow$  & RMSE $\downarrow$ \\
\midrule
AKAZE
& 10.103 $\pm$ 16.763 & 11.704 $\pm$ 19.743 & 1.572 $\pm$ 0.660 & 2.479 $\pm$ 2.295 \\
KAZE
& 37.618 $\pm$ 22.656 & 43.058 $\pm$ 25.450 & 1.442 $\pm$ 0.630 & 3.655 $\pm$ 3.368 \\
AKAZE + BRISK
& 42.866 $\pm$ 23.073 & 47.927 $\pm$ 25.414 & 1.152 $\pm$ 0.230 & 1.735 $\pm$ 0.697 \\
AKAZE + KAZE
& 21.433 $\pm$ 23.693 & 25.508 $\pm$ 27.314 & 1.350 $\pm$ 0.252 & 2.805 $\pm$ 2.063 \\
AKAZE + SIFT
& 25.950 $\pm$ 26.961 & 30.114 $\pm$ 29.092 & 1.497 $\pm$ 0.238 & 3.262 $\pm$ 1.979 \\
AKAZE + SuperPoint
& 1.990 $\pm$ 0.198 & 2.427 $\pm$ 0.465 &